<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

## Import

In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [ ]:
# 모르겠다, 시간 없으니 통과.

In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


## TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object

response = requests.get( static_url )

if response.status_code == 200:
    print(f'html content : {response.content[:300]}')
else:
    print('failed.')

html content : b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vect'


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

soup = BeautifulSoup( response.text, 'html.parser')

In [8]:
# test
type( soup )

bs4.BeautifulSoup

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [11]:
# Use soup.title attribute

print(f'page title : {soup.title}')

page title : <title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


## TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


먼저 위키 페이지의 모든 테이블을 찾아보겠습니다. BeautifulSoup에 대한 기억을 되새기려면 이 실습 마지막 부분에 있는 외부 참조 링크를 확인하세요.

In [12]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# 요소 유형 `table`을 사용하여 BeautifulSoup 객체에서 find_all 함수를 사용합니다.
# `html_tables`라는 목록에 결과를 할당합니다.

html_tables = soup.find_all('table')

In [13]:
# test
type( html_tables )

bs4.element.ResultSet

Starting from the third table is our target table contains the actual launch records.


3번째 table : 2022 ?

In [ ]:
# 출력 결과가 너무 길어서, 지움.

In [ ]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

In [18]:
# test
type( first_launch_table )

bs4.element.Tag

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [20]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

# first_launch_table의 `th` 요소와 함께 find_all() 함수를 적용합니다.
# 각 요소를 반복하고 제공된 extract_column_from_header()를 적용하여 열 이름을 얻습니다.
# 비어 있지 않은 열 이름(`이름이 None이 아니고 len(name) > 0인 경우`)을 column_names라는 목록에 추가합니다.

column_names = []

# 첫 번째 테이블의 모든 th 요소를 찾아서 열 이름을 추출
column_headers = first_launch_table.find_all( 'th' )

# 각 열 이름을 추출하여 column_names 목록에 추가
for header in column_headers:
    # th 요소에서 텍스트 추출
    name = header.text.strip()
    
    if name and len(name)>0:
        column_names.append( name )

Check the extracted column names


In [21]:
print(column_names)

['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


이전 작업에서 추출한 열 이름의 키를 사용하여 빈 사전을 생성합니다. 나중에 이 사전은 Pandas 데이터 프레임으로 변환됩니다.

In [84]:
# 위에서 추출한 컬럼명이 아래와 다른 것이 있어서, 수동으로 하자.
column_names_2 = [  'Flight No.', 'Date andtime (UTC)'
#                 , 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass'
                  , 'Version Booster', 'Launch Site', 'Payload', 'Payload Mass'
#                 , 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding'
                  , 'Orbit', 'Customer', 'Launch Outcome', 'Booster Landing'
                 ]

In [90]:
# 이런 방법도 있군.
#launch_dict = dict.fromkeys(column_names)
launch_dict = dict.fromkeys(column_names_2)

# Remove an irrelvant column
# del launch_dict['Date and time ( )']
del launch_dict['Date andtime (UTC)']

# Let's initial the launch_dict with each value to be an empty list
# 스페이스 있네?
launch_dict['Flight No.'] = []
#launch_dict['Launch site'] = []
launch_dict['Launch Site'] = []
launch_dict['Payload'] = []
#launch_dict['Payload mass'] = []
launch_dict['Payload Mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
#launch_dict['Launch outcome'] = []
launch_dict['Launch Outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
#launch_dict['Booster landing']=[]
launch_dict['Booster Landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

# 일관선 없는 대소문자 때문에 Error 나서, 모두 대문자 시작으로 수정 함.

In [91]:
# test
launch_dict

{'Flight No.': [],
 'Version Booster': [],
 'Launch Site': [],
 'Payload': [],
 'Payload Mass': [],
 'Orbit': [],
 'Customer': [],
 'Launch Outcome': [],
 'Booster Landing': [],
 'Date': [],
 'Time': []}

In [92]:
# test
len( launch_dict )

11

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


일반적으로 Wiki 페이지의 HTML 테이블에는 참조 링크 B0004.1[8], 누락된 값 N/A[e], 일관되지 않은 형식 등과 같은 예상치 못한 주석과 기타 유형의 노이즈가 포함될 가능성이 있습니다.

구문 분석 프로세스를 단순화하기 위해 launch_dict를 작성하는 데 도움이 되는 불완전한 코드 조각을 아래에 제공했습니다. TODO를 사용하여 다음 코드 조각을 완성하거나 모든 실행 테이블을 구문 분석하는 자체 논리를 작성하도록 선택할 수 있습니다.

In [93]:
# 디버그 출력 여부 (1:True, 0:False)
# debug_tf = 1
debug_tf = 0

In [94]:
extracted_row = 0

#Extract each table 
# 테이블은 "wikitable plainrowheaders collapsible" 클래스를 가지고 있어야 합니다.
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        # 각 행에서 첫 번째 열 (th 요소)을 확인하여 해당 열이 "Flight No." (로켓 발사 번호)와 관련된 것인지를 확인
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row = rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            # TODO: `Flight No.` 키를 사용하여 launch_dict에 flight_number를 추가합니다.
            #launch_dict['Flight No.'] = flight_number
            launch_dict['Flight No.'].append( flight_number )
            #print(flight_number)
            if debug_tf == 1:  print('')
            if debug_tf == 1:  print('flight_number : ', flight_number)
            
            # 날짜와 시간을 추출하여 리스트로 반환
            datatimelist = date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            # 반환된 날짜 정보에서 ',' 문자를 제거
            date = datatimelist[0].strip(',')
            #launch_dict['Date'] = date
            launch_dict['Date'].append( date )
            #print(date)
            if debug_tf == 1:  print('date : ', date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #launch_dict['Time'] = time
            launch_dict['Time'].append( time )
            #print(time)
            if debug_tf == 1:  print('time : ', time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            #launch_dict['Version Booster'] = bv
            launch_dict['Version Booster'].append( bv )
            #print(bv)
            if debug_tf == 1:  print('bv : ', bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #launch_dict['Launch Site'] = launch_site
            #launch_dict['Launch Site'].append( launch_site )
            launch_dict['Launch Site'].append( launch_site )  # 대소문자 에러!
            #print(launch_site)
            if debug_tf == 1:  print('launch_site : ', launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #launch_dict['Payload'] = payload
            launch_dict['Payload'].append( payload )
            #print(payload)
            if debug_tf == 1:  print('payload : ', payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #launch_dict['Payload mass'] = payload_mass
            #launch_dict['Payload mass'].append( payload_mass )
            launch_dict['Payload Mass'].append( payload_mass )
            #print(payload)
            if debug_tf == 1:  print('payload_mass : ', payload_mass)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #launch_dict['Orbit'] = orbit
            launch_dict['Orbit'].append( orbit )
            #print(orbit)
            if debug_tf == 1:  print('orbit : ', orbit)
            
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            #customer = row[6].a.string
            #launch_dict['Customer'] = customer
            #print(customer)
            #if debug_tf == 1:  print('customer : ', customer)
            
            # 위 코드에서 아래 에러 남
            # AttributeError: 'NoneType' object has no attribute 'string'
            
            # 수정 함.
            # if len( row[6] ) > 0:
            # if row[6] is not None:
            if row[6].a is not None:  # html a 태그 없어서 에러 나는 듯.
                if debug_tf == 1:  print('row[6] : ', row[6])
                customer = row[6].a.string
                #launch_dict['Customer'] = customer
                launch_dict['Customer'].append( customer )
                if debug_tf == 1:  print('customer : ', customer)
            else:
                #launch_dict['Customer'] = None            
                launch_dict['Customer'].append( None )
            
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #launch_dict['Launch outcome'] = launch_outcome
            #launch_dict['Launch outcome'].append( launch_outcome )
            launch_dict['Launch Outcome'].append( launch_outcome )
            #print(launch_outcome)
            if debug_tf == 1:  print('launch_outcome : ', launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #launch_dict['Booster landing'] = booster_landing
            #launch_dict['Booster landing'].append( booster_landing )
            launch_dict['Booster Landing'].append( booster_landing )
            #print(booster_landing)
            if debug_tf == 1:  print('booster_landing : ', booster_landing)
            
print('Finished.')

Finished.


In [97]:
# test
len( launch_dict )

11

G

위의 파이썬 코드는 BeautifulSoup을 사용하여 웹 페이지에서 데이터를 추출하고 이를 정리하여 launch 정보를 추출하는 코드입니다. 코드를 세분화하여 설명하겠습니다.

1. `for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):`: 웹 페이지에서 `<table>` 태그 중에서 클래스가 "wikitable plainrowheaders collapsible"인 테이블을 찾아 순회합니다. 이러한 테이블에는 SpaceX의 로켓 발사 정보가 포함되어 있습니다.

2. `for rows in table.find_all("tr"):`: 각 테이블에서 행(`<tr>`)을 찾아 순회합니다.

3. `if rows.th:`: 각 행에서 첫 번째 열(`<th>`)이 있는지 확인합니다.

4. `flight_number=rows.th.string.strip()`: 첫 번째 열(`<th>`)의 내용을 문자열로 가져옵니다. 이것이 로켓 발사 번호인지 확인하기 위해 `.isdigit()` 함수를 사용하여 숫자인지 확인합니다.

5. `datatimelist=date_time(row[0])`: 첫 번째 열에서 날짜와 시간 정보를 추출하기 위해 `date_time` 함수를 호출합니다. 이 함수는 날짜와 시간을 추출하여 리스트로 반환합니다.

6. `date = datatimelist[0].strip(',')`: 반환된 날짜 정보에서 ',' 문자를 제거하고 `date` 변수에 저장합니다.

7. `time = datatimelist[1]`: 반환된 시간 정보를 `time` 변수에 저장합니다.

8. `bv=booster_version(row[1])`: 두 번째 열에서 부스터 버전 정보를 추출하기 위해 `booster_version` 함수를 호출합니다. 이 함수는 부스터 버전 정보를 반환하거나 `<a>` 태그로 묶여 있는 경우 해당 링크 텍스트를 반환합니다.

9. `launch_site = row[2].a.string`: 세 번째 열에서 발사 사이트 정보를 추출합니다.

10. `payload = row[3].a.string`: 네 번째 열에서 페이로드 정보를 추출합니다.

11. `payload_mass = get_mass(row[4])`: 다섯 번째 열에서 페이로드 질량 정보를 추출하기 위해 `get_mass` 함수를 호출합니다. 이 함수는 질량 정보를 추출하여 반환합니다.

12. `orbit = row[5].a.string`: 여섯 번째 열에서 궤도 정보를 추출합니다.

13. `customer = row[6].a.string`: 일곱 번째 열에서 고객 정보를 추출합니다.

14. `launch_outcome = list(row[7].strings)[0]`: 여덟 번째 열에서 발사 결과 정보를 추출합니다.

15. `booster_landing = landing_status(row[8])`: 아홉 번째 열에서 부스터 착륙 정보를 추출하기 위해 `landing_status` 함수를 호출합니다. 이 함수는 착륙 정보를 반환합니다.

16. 주석의 'TODO' 지시문에 따라 각 정보를 `launch_dict`에 추가합니다.

이러한 방식으로 각 행에서 필요한 정보를 추출하고 `launch_dict`에 저장하여 정리합니다.

G

아래 코드는 주어진 HTML 표에서 "Flight No." (로켓 발사 번호) 값을 추출하고, 해당 값에서 앞 뒤의 공백을 제거하는 부분입니다.

```python
flight_number = rows.th.string.strip()
```

여기에서:

- `rows.th`는 현재 표의 열에서 첫 번째 셀 (`<th>` 태그)을 나타냅니다.
- `rows.th.string`은 해당 셀의 텍스트 내용을 나타냅니다. 즉, "Flight No." 문자열 또는 해당 로켓 발사 번호입니다.
- `strip()` 함수는 문자열에서 앞뒤의 공백을 제거하는 메서드입니다. 따라서 `flight_number` 변수에는 "Flight No." 문자열 또는 실제 로켓 발사 번호가 들어갑니다.

이 코드를 통해 현재 행의 첫 번째 셀에서 로켓 발사 번호를 추출하고 이를 `flight_number` 변수에 저장합니다. 이후에 이 값을 사용하여 `launch_dict`에 로켓 발사 번호를 추가하는 데 활용됩니다.

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [98]:
df = pd.DataFrame({ key:pd.Series(value)  for key, value in launch_dict.items() })

df.head(3)

,Flight No.,Version Booster,Launch Site,Payload,Payload Mass,Orbit,Customer,Launch Outcome,Booster Landing,Date,Time
0,1,F9 v1.0B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,Failure,4 June 2010,18:45
1,2,F9 v1.0B0004.1,CCAFS,Dragon,0,LEO,NASA,Success,Failure,8 December 2010,15:43
2,3,F9 v1.0B0005.1,CCAFS,Dragon,525 kg,LEO,NASA,Success,Not attempted\n,22 May 2012,07:44


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>

In [99]:
df.to_csv('dataset_part_2(web_scraped).csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
